<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/SwissLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/EngCorpus.pkl'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/tokenize_group_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/mask_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/knowledge_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/file_output_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/dict_group_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/evaluator/evaluator.py'


--2023-08-03 06:19:21--  https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/EngCorpus.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 412351 (403K) [text/plain]
Saving to: ‘EngCorpus.pkl’

EngCorpus.pkl       100%[===================>] 402.69K  --.-KB/s    in 0.04s   

2023-08-03 06:19:21 (11.1 MB/s) - ‘EngCorpus.pkl’ saved [412351/412351]

--2023-08-03 06:19:21--  https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/tokenize_group_layer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiti

In [2]:
# upload logs
!wget 'https://raw.githubusercontent.com/logpai/loghub/master/HDFS/HDFS_2k.log'

--2023-08-03 06:19:23--  https://raw.githubusercontent.com/logpai/loghub/master/HDFS/HDFS_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285848 (279K) [text/plain]
Saving to: ‘HDFS_2k.log’

HDFS_2k.log         100%[===================>] 279.15K  --.-KB/s    in 0.03s   

2023-08-03 06:19:23 (8.61 MB/s) - ‘HDFS_2k.log’ saved [285848/285848]



In [3]:
import sys
sys.path.append('/content/') #to be able for importing libarary from this address

In [4]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541529 sha256=7ba6962f9712d498560020f17be69bc041727a10bdf58965313de6ec7ba968ab
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [5]:
from file_output_layer import FileOutputLayer
from knowledge_layer import KnowledgeGroupLayer
from mask_layer import MaskLayer
from tokenize_group_layer import TokenizeGroupLayer
from dict_group_layer import DictGroupLayer
import evaluator

import os
import re
import string
import hashlib
from datetime import datetime
from tqdm import tqdm
import pandas as pd

In [6]:
input_dir = '/content/' # The input directory of log file
output_dir = 'LogParserResult/' # The output directory of parsing results

In [7]:
log_file = 'HDFS_2k.log'
log_format =  '<Date> <Time> <Pid> <Level> <Component>: <Content>'
regex = [r'blk_-?\d+', r'(\d+\.){3}\d+(:\d+)?']


In [8]:
setting = {
        'log_file': 'HDFS_2k.log',
        'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>',
        'regex': [r'blk_-?\d+', r'(\d+\.){3}\d+(:\d+)?']
        }

In [9]:
def generate_logformat_regex(logformat):
    """ Function to generate regular expression to split log messages
    """
    headers = []
    splitters = re.split(r'(<[^<>]+>)', logformat)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += '(?P<%s>.*?)' % header
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex



In [10]:
def load_logs(log_file, regex, headers):
    """ Function to transform log file to dataframe
    """
    log_messages = dict()
    linecount = 0
    with open(log_file, 'r') as fin:
        for line in tqdm(fin.readlines(), desc='load data'):

            try:
                linecount += 1
                match = regex.search(line.strip())
                message = dict()
                for header in headers:
                    message[header] = match.group(header)
                message['LineId'] = linecount
                log_messages[linecount] = message
            except Exception as e:
                pass
    return log_messages

In [11]:
headers, regex = generate_logformat_regex(setting['log_format'])
log_messages = load_logs('/content/HDFS_2k.log', regex, headers)
# preprocess layer
log_messages = KnowledgeGroupLayer(log_messages).run()
# tokenize layer
log_messages = TokenizeGroupLayer(log_messages, rex=setting['regex']).run()
# dictionarize layer and cluster by wordset
dict_group_result = DictGroupLayer(log_messages, '/content/EngCorpus.pkl').run()
# apply LCS and prefix tree
results, templates = MaskLayer(dict_group_result).run()

priori knowledge preprocess: 100%|██████████| 2000/2000 [00:00<00:00, 21620.12it/s]


Knowledge group layer finished.


tokenization: 100%|██████████| 2000/2000 [00:00<00:00, 22337.22it/s]


Tokenization layer finished.


group by wordset: 100%|██████████| 2000/2000 [00:00<00:00, 447655.05it/s]


After Dictionary Group, total: 14 bin(s)


generate output: 100%|██████████| 14/14 [00:00<00:00, 3094.12it/s]

After mask layer finish, tot, total: 14 bin(s)


In [12]:
FileOutputLayer(log_messages, '/content/results/HDFS', templates, ['LineId'] + headers).run()